# Assignment 4 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [4]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;
import plotnine as pn;
from datetime import datetime;
from dateutil.relativedelta import relativedelta
from sklearn.neighbors import NearestNeighbors;
from sklearn.linear_model import LinearRegression;

In [6]:
## From Midterm
data = pd.read_csv("SPX_W_incomplete.csv")

#generate x values
date_grid = []
for i in np.arange(52):
    thisdate = ( datetime.strptime('2023-09-25', '%Y-%m-%d') + relativedelta(weeks = -int(i)) ).strftime('%F')
    date_grid.append(thisdate)
date_grid = np.asarray(date_grid, dtype=np.datetime64)
date_grid = np.flip(date_grid)

#fix date column in dataframe
data_date_Col = [];
for row in data["Date"]:
   thisdate = datetime.strptime(row[0:10], '%Y-%m-%d').strftime('%F')
   data_date_Col.append(thisdate)
data_date_Col = np.asarray(data_date_Col, dtype = np.datetime64)

data['Date'] = data_date_Col

## Part [a]

In [9]:
K = 15;
#target weeks
weeks = np.arange(0, 52)
weekssq = weeks**2

##Find Nearest Neighbors
nn = NearestNeighbors(n_neighbors = K, p = 1).fit(np.asarray(data['WeekNumber']).reshape(-1, 1));
kn = nn.kneighbors(weeks.reshape(-1, 1), K, return_distance= True)

#find max distances - using the 8th nearest neighbor, neighbors in ascending order of distance
distances = kn[0]
maxdistances = np.expand_dims(kn[0][:,14], axis = 1)

#Get scaled distances
scaled_dist = distances/maxdistances

In [ ]:
#tri-cube function
tricube = lambda distance_vec: np.power(1 - np.power(np.abs(distance_vec), 3), 3)

weights = tricube(scaled_dist)

In [30]:
week_array = np.asarray(data['WeekNumber']).reshape(-1,1);
extra_array = [];
extra_neighbors_array = [];
for i in range(len(weeks)):
    rn = NearestNeighbors(radius = float(maxdistances[i]), p = 1).fit(week_array);
    kn = rn.radius_neighbors(weeks[i].reshape(-1,1), radius = float(maxdistances[i]), return_distance= True)
    if len(kn[0][0]) > K:
        extra_array.append(i)
        extra_neighbors_array.append(kn[0][0])
extra_neighbors_array = np.asarray(extra_neighbors_array);

C:\Users\Dennis Goldenberg\AppData\Local\Temp\ipykernel_12088\414734828.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
C:\Users\Dennis Goldenberg\AppData\Local\Temp\ipykernel_12088\414734828.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
C:\Users\Dennis Goldenberg\AppData\Local\Temp\ipykernel_12088\414734828.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
C:\Users\Dennis Goldenberg\AppData\Local\Temp\ipykernel_12088\414734828.py:6: DeprecationWarning: Conve

In [13]:
extras = np.asarray([K + 1] * len(extra_array))
extra_data = {"Week Number": extra_array, "Date": date_grid[extra_array], "Number of Neighbors": extras}
extra_df = pd.DataFrame(extra_data)
extra_df

,Week Number,Date,Number of Neighbors
0,14,2023-01-09,16
1,17,2023-01-30,16
2,23,2023-03-13,16
3,24,2023-03-20,16
4,27,2023-04-10,16
5,29,2023-04-24,16
6,32,2023-05-15,16
7,35,2023-06-05,16


## Part [b]

## Part [c]